<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/Section2_project%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install riotwatcher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 3.2 MB/s 


In [ ]:
def getAPIkey():
    f = open('./riot_api.txt', 'r')
    return f.read()

In [2]:
import pandas as pd
import requests
import time
import json
from pandas import json_normalize

In [1]:
# api_key = getAPIkey()
api_key = 'RGAPI-2d27cefe-76e4-4b9f-a814-846ed26b7df1'
REGION_SET= 'kr'
my_summoner_name= '돈까스사주세요'

In [3]:
request_header ={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}
summoner_api = 'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/'+my_summoner_name+\
    '?api_key='+api_key

In [4]:
r = requests.get(summoner_api, headers=request_header)
pd.DataFrame(r.json(), index=[0]).T

,0
id,FetGOBC727ubLYe9DspT37HhJ8V2yphSj1QsnNCZnit6PQ
accountId,3oGFKpfFSDmDpDq7L45ZfE7gy9G_rXPQzqRDzajIsFWr
puuid,VTXPvggnkk3U4tVfyV4FZgBZoHnjjMmMber4ZsY3rXstST...
name,돈까스사주세요
profileIconId,4794
revisionDate,1669515976000
summonerLevel,248


In [5]:
puuid = r.json()['puuid']

request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

match_url = 'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/'\
    +puuid+'/ids?start=0&count=30&api_key='+api_key

r = requests.get(match_url, headers=request_header)
r.json()

['KR_6234854391',
 'KR_6234829189',
 'KR_6234622932',
 'KR_6234521384',
 'KR_6234445601',
 'KR_6234248291',
 'KR_6233949013',
 'KR_6233825487',
 'KR_6233773134',
 'KR_6233681631',
 'KR_6233470822',
 'KR_6233352012',
 'KR_6233276446',
 'KR_6233225740',
 'KR_6233059919',
 'KR_6233052447',
 'KR_6232503455',
 'KR_6232187142',
 'KR_6231979288',
 'KR_6231768656',
 'KR_6231312734',
 'KR_6231031319',
 'KR_6230775377',
 'KR_6230666167',
 'KR_6229904271',
 'KR_6229809077',
 'KR_6229755889',
 'KR_6229584243',
 'KR_6229417567',
 'KR_6229407479']

In [6]:
game_id=r.json()[0] # 최근 30게임 중 가장 최근 1게임

request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

match_url = 'https://asia.api.riotgames.com/tft/match/v1/matches/' + game_id + '?api_key='+api_key

r = requests.get(match_url, headers=request_header)
df= pd.DataFrame(r.json())
df

,metadata,info
data_version,5,NaN
match_id,KR_6234854391,NaN
participants,[EmY1aN1dCBA5cf4G5y2CKdbcgEx9Hl8-AhJIWyVD4xfb9...,"[{'augments': ['TFT6_Augment_RichGetRicher', '..."
game_datetime,NaN,1669515966239
game_length,NaN,2111.860352
game_version,NaN,Version 12.22.479.5277 (Nov 11 2022/10:35:17) ...
queue_id,NaN,1100
tft_game_type,NaN,standard
tft_set_core_name,NaN,TFTSet7_2
tft_set_number,NaN,7


In [7]:
df = pd.DataFrame(df['info']['participants'])

In [86]:
df[isMe].index

Int64Index([5], dtype='int64')

In [87]:
isMe = df.puuid == puuid
idx = df[isMe].index
df[isMe].augments

5    [TFT6_Augment_RichGetRicher, TFT7_Augment_Drag...
Name: augments, dtype: object

In [62]:
df[isMe].augments.values

array([list(['TFT6_Augment_RichGetRicher', 'TFT7_Augment_DragonmancerInTraining', 'TFT6_Augment_JeweledLotus'])],
      dtype=object)

In [95]:
isMe = df.puuid == puuid
df_me = pd.DataFrame({
    'placement': df[isMe].loc[idx, 'placement'], 
    'augments': df[isMe].loc[idx, 'augments'],
    'traits': df[isMe].loc[idx, 'traits'],
    'units': df[isMe].loc[idx, 'units'] }).reset_index(drop=True)

In [91]:
df_me

,placement,augments,traits,units
0,1,"[TFT6_Augment_RichGetRicher, TFT7_Augment_Drag...","[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Sett', 'itemNames': ['..."


In [106]:
# placement : 해당 경기의 '돈까스사주세요' 유저의 최종순위

In [102]:
# augments: 선택한 증강체
pd.DataFrame(df_me.augments.loc[0]).T

,0,1,2
0,TFT6_Augment_RichGetRicher,TFT7_Augment_DragonmancerInTraining,TFT6_Augment_JeweledLotus


In [105]:
# traints: 조합 시너지 - 시너지 이름, 적용 유닛수, 시너지 등급(브론즈1/실버2/골드3/플래티넘4), tier_total?
pd.DataFrame(df_me.traits.loc[0])

,name,num_units,style,tier_current,tier_total
0,Set7_Bard,1,3,1,1
1,Set7_Dragonmancer,6,3,3,4
2,Set7_Guild,1,1,1,8
3,Set7_Jade,1,0,0,4
4,Set7_Lagoon,1,0,0,4
5,Set7_Mystic,1,0,0,4
6,Set7_Ragewing,1,0,0,4
7,Set7_Shimmerscale,1,0,0,4
8,Set7_Swiftshot,1,0,0,4
9,Set7_Tempest,2,1,1,4


In [107]:
# units: 사용한 유닛들 - 유닛이름, 착용아이템, 착용아이템의 고유번호, 유닛희귀도, 1성/2성/3성 등급
pd.DataFrame(df_me.units.loc[0])

,character_id,itemNames,items,name,rarity,tier
0,TFT7_Sett,[TFT_Item_RedBuff],[57],,0,3
1,TFT7_Karma,"[TFT_Item_Morellonomicon, TFT_Item_InfinityEdg...","[37, 19, 24]",,0,3
2,TFT7_Ezreal,"[TFT_Item_SeraphsEmbrace, TFT_Item_MadredsBloo...","[44, 12, 2426]",,0,3
3,TFT7_Kaisa,[],[],,1,3
4,TFT7_LeeSin,"[TFT_Item_GuardianAngel, TFT_Item_InfinityEdge]","[94, 19]",,2,3
5,TFT7b_Volibear,[],[],,2,2
6,TFT7b_Volibear,[TFT_Item_GuardianAngel],[94],,2,2
7,TFT7_Bard,[],[],,6,1


In [112]:
df[isMe]['companion'].loc[5]

{'content_ID': '820cd738-5bff-4208-9de6-6b4558fec56d',
 'item_ID': 13017,
 'skin_ID': 17,
 'species': 'PetAkaliDragon'}

In [113]:
df = df.sort_values('placement')

In [114]:
df.set_index('placement', inplace=True)

In [115]:
df

,augments,companion,gold_left,last_round,level,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
placement,,,,,,,,,,,
1,"[TFT6_Augment_RichGetRicher, TFT7_Augment_Drag...",{'content_ID': '820cd738-5bff-4208-9de6-6b4558...,12,38,8,2,VTXPvggnkk3U4tVfyV4FZgBZoHnjjMmMber4ZsY3rXstST...,2103.725342,214,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Sett', 'itemNames': ['..."
2,"[TFT6_Augment_RichGetRicher, TFT7_Augment_Jade...",{'content_ID': '1463333c-3219-4e5b-b820-5e4b48...,0,38,9,1,EmY1aN1dCBA5cf4G5y2CKdbcgEx9Hl8-AhJIWyVD4xfb9w...,2103.725342,104,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Wukong', 'itemNames': ..."
3,"[TFT6_Augment_SunfireBoard, TFT6_Augment_Ascen...",{'content_ID': 'f56ecac7-ed16-483d-8b9c-2bbced...,10,33,7,2,qbm-L2Qf4acS_ggSy4Ij3WyDhL-8NvPcLAwbIJFW2r7lVb...,1813.669434,89,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
4,"[TFT6_Augment_ThrillOfTheHunt2, TFT7_Augment_P...",{'content_ID': '8c27f67c-90b3-4355-9e90-6e9232...,0,31,8,2,_AjnrIRLWhpIgq23Pa6abBj8GHy-OUE9M510RRuAsi988N...,1721.090454,120,"[{'name': 'Set7_Assassin', 'num_units': 4, 'st...","[{'character_id': 'TFT7_Qiyana', 'itemNames': ..."
5,"[TFT6_Augment_TomeOfTraits1, TFT7_Augment_Drag...",{'content_ID': '97179efb-3af4-4221-b42a-9e7b88...,31,31,7,0,qa5SW0l7Ncm5WJ1ZMmDBzWIGkLIE2jtISETOJkVq784s_D...,1724.035645,68,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Sett', 'itemNames': ['..."
6,"[TFT6_Augment_JeweledLotus, TFT6_Augment_Compo...",{'content_ID': 'b5ffc1a8-89fc-4fd6-8c6f-afda65...,44,30,7,0,B1B04BM8mcbcIVJ5CAxrh03s4yJFptoa_HdTAc6ybikQVz...,1662.010376,85,"[{'name': 'Set7_Astral', 'num_units': 5, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
7,"[TFT7_Augment_Bloodlust1, TFT7_Augment_ScopedW...",{'content_ID': '20649e3b-c1c0-4300-8f0e-4a259c...,21,26,6,0,KofeBCRBpuGa3htnGICVevIDipJxoy0QiromUXLMdxk4UU...,1426.900146,44,"[{'name': 'Set7_Assassin', 'num_units': 1, 'st...","[{'character_id': 'TFT7_Yone', 'itemNames': ['..."
8,"[TFT6_Augment_RichGetRicher, TFT6_Augment_Seco...",{'content_ID': '5c4177c8-1b87-43be-9a69-4113eb...,28,26,8,0,VNzPf5saIxhFwvB8xFp7JbSZaLulqA1fjO9zhlwXZJEJpb...,1435.827759,46,"[{'name': 'Set7_Astral', 'num_units': 3, 'styl...","[{'character_id': 'TFT7_Zyra', 'itemNames': []..."


In [ ]:
df.loc[8, 'augments']

['TFT6_Augment_TomeOfTraits1',
 'TFT6_Augment_VerdantVeil',
 'TFT6_Augment_ThrillOfTheHunt2']

In [ ]:
df[df['puuid'] == puuid]

,augments,companion,gold_left,last_round,level,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
placement,,,,,,,,,,,
8,"[TFT6_Augment_TomeOfTraits1, TFT6_Augment_Verd...",{'content_ID': '820cd738-5bff-4208-9de6-6b4558...,10,24,6,0,VTXPvggnkk3U4tVfyV4FZgBZoHnjjMmMber4ZsY3rXstST...,1456.276123,31,"[{'name': 'Set7_Astral', 'num_units': 3, 'styl...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."


In [ ]:
# 각각의 컬럼이 의미하는 것을 파악해보자
# augments : 선택한 증강체
# companion

In [ ]:
pd.DataFrame(df.traits[0])
### name : 해당 유저가 사용한 시너지 표기
## Set7_ : 시즌 7

## 단독 시너지
# Bard : 음유시인(바드)
# Spell Thief : 주문도둑(조이)
# Starcaller : 별부르미(소라카)

## 직업 시너지
# Bruiser : 난동꾼
# Dragon : 용
# Dragonmancer : 용술사
# Guardian : 수호자
# Mystic : 신비술사
# Swiftshot : 민첩사수
# Mage : 요술사
# Warrior ; 전사
# Evoker : 일깨우는자
# Cavalier : 기병대
# Cannoneer : 포병대
# Assassin : 암살자

## 종족 시너지
# Whispers : 속삭임
# Tempest : 폭풍
# Scalescorn : 비늘혐오자
# Guild : 길드
# Mirage : 신기루
# Lagoon : 석호
# Darkflight : 어둠강림
# Ragewing : 분노날개
# Astral : 별
# Jade : 비취
# Shimmerscale : 빛비늘
# Prodigy : 신동(뇸뇸이)
# Monolith : 거석(테라)

,name,num_units,style,tier_current,tier_total
0,Set7_Bard,1,3,1,1
1,Set7_Bruiser,1,0,0,4
2,Set7_Dragon,2,2,2,6
3,Set7_Dragonmancer,1,0,0,4
4,Set7_Guardian,2,1,1,4
5,Set7_Guild,1,1,1,8
6,Set7_Jade,1,0,0,4
7,Set7_Monolith,3,3,1,1
8,Set7_Mystic,1,0,0,4
9,Set7_Ragewing,1,0,0,4


In [ ]:
def league_summoner(key, tier, country='kr',):
    request =requests.get(f'https://{country}.api.riotgames.com/tft/league/v1/{tier}?api_key={key}')
    return json.loads(request.content)

In [ ]:
ch_summoner = league_summoner(api_key, tier='challenger')
ch_summoner

{'tier': 'CHALLENGER',
 'leagueId': '946274f2-16b6-376e-8d60-e07ef8f06a7f',
 'queue': 'RANKED_TFT',
 'name': "Orianna's Dragons",
 'entries': [{'summonerId': 'ZE0kuJXyyCEvdzCB_1bER7GgMW5MoQ3fVdKm7igcyKZY0Dg',
   'summonerName': '페이도',
   'leaguePoints': 1646,
   'rank': 'I',
   'wins': 186,
   'losses': 81,
   'veteran': True,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'Xn63kjwkKUv6q6F7XuD3fKDwtGamqJ7mOq36wjrPDdKsjA',
   'summonerName': '이세사범',
   'leaguePoints': 1000,
   'rank': 'I',
   'wins': 126,
   'losses': 91,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'goe0j8G02-cvvoiQFcKj5W_pPIBIS4d3NKNRpiApAjREVRo',
   'summonerName': '장봄준 fan',
   'leaguePoints': 1144,
   'rank': 'I',
   'wins': 199,
   'losses': 171,
   'veteran': False,
   'inactive': False,
   'freshBlood': False,
   'hotStreak': False},
  {'summonerId': 'Y8meZI_sW0BCJ43LYr9_QzdVQcTbuzgH6YcO32KlNZksiKA',
   'summon

In [ ]:
ch_summoner = json_normalize(ch_summoner['entries'])
ch_summoner

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,ZE0kuJXyyCEvdzCB_1bER7GgMW5MoQ3fVdKm7igcyKZY0Dg,페이도,1646,I,186,81,True,False,False,False
1,Xn63kjwkKUv6q6F7XuD3fKDwtGamqJ7mOq36wjrPDdKsjA,이세사범,1000,I,126,91,False,False,False,False
2,goe0j8G02-cvvoiQFcKj5W_pPIBIS4d3NKNRpiApAjREVRo,장봄준 fan,1144,I,199,171,False,False,False,False
3,Y8meZI_sW0BCJ43LYr9_QzdVQcTbuzgH6YcO32KlNZksiKA,KT DY,914,I,166,125,True,False,False,False
4,3k8acfh5IPBXP3xeYk8OIJvx2FeCBBXKUYRRAqQxCvoEHp8,길 다,999,I,150,106,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
295,1bvplhZsZjbwQclvR3kisT1Ii6FMnVv8Jfeg__zv2sIe0v0,TwTF 근열맨,1438,I,246,161,True,False,False,True
296,_Gly8q0N6-2ptBRemUcJtCRy3niPmuWRkozZfMBWg3ryxx...,산꼬대,967,I,106,82,False,False,True,True
297,Fg0JMXvZw07820KfBiqWQb1NwrI9F6uTqFVM-22VN6_48_0,Oriente Verano,1093,I,202,144,True,False,False,False
298,mfWehEZxLMgd9iLtUZbzfbq7QJcJvAWgoYa-IW4R0e6YNA,지그미야,1037,I,156,126,True,False,False,False


In [ ]:
summonerId = ch_summoner['summonerId']
summonerId

0        ZE0kuJXyyCEvdzCB_1bER7GgMW5MoQ3fVdKm7igcyKZY0Dg
1         Xn63kjwkKUv6q6F7XuD3fKDwtGamqJ7mOq36wjrPDdKsjA
2        goe0j8G02-cvvoiQFcKj5W_pPIBIS4d3NKNRpiApAjREVRo
3        Y8meZI_sW0BCJ43LYr9_QzdVQcTbuzgH6YcO32KlNZksiKA
4        3k8acfh5IPBXP3xeYk8OIJvx2FeCBBXKUYRRAqQxCvoEHp8
                             ...                        
295      1bvplhZsZjbwQclvR3kisT1Ii6FMnVv8Jfeg__zv2sIe0v0
296    _Gly8q0N6-2ptBRemUcJtCRy3niPmuWRkozZfMBWg3ryxx...
297      Fg0JMXvZw07820KfBiqWQb1NwrI9F6uTqFVM-22VN6_48_0
298       mfWehEZxLMgd9iLtUZbzfbq7QJcJvAWgoYa-IW4R0e6YNA
299      iKkU3cEGjL8JQioBO1ivsPLLmxudpqU-mFOaqhQ3sskTN1U
Name: summonerId, Length: 300, dtype: object

In [ ]:
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

def summoner_info(summonerId , key, country='kr'):
    request =requests.get(f'https://{country}.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}?api_key={key}', headers=request_header)
    return json.loads(request.content)
from time import sleep



def summoner_puuid(summonerId):
    summ_puuid = pd.Series([])
    l = 100
    for i in summonerId:
        l += 1
        if l%100 == 0:
            sleep(120)
        si = summoner_info(i,api_key,'kr') 
        si = json_normalize(si)
        summ_puuid = summ_puuid.append(si['puuid'],ignore_index = True)
        
    return summ_puuid
puuid = summoner_puuid(summonerId)
puuid

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()


0      rk9gcmTmQOHdzPP9ANWHvjB0lmnt0zinvTAVQEgvTNQafk...
1      pFzvA0I0xYkLAJv6hBrwMi-k5GeayGgD7XjPfvNvN75yZv...
2      vnXlUq8lf62uLcBEm4iqKEVt2eFDXa6xbrcgmDaexACM9E...
3      hmiQGBTmq6f0Jm4GNYBGR6IC18xlMSGX5SiJOwdHhbJIqk...
4      2xwCj4UlDQaaqKqk8-Qb1HEyfqW08hJb2ZqfVmMSI_oNKl...
                             ...                        
295    t1_RuefvjeSnVrg0gTmg4-KkSRy8rAeH8CCZVpjQWj2wDp...
296    j4wS7ToUvpSdmkReE1hOcYTKcF4DeaNfF94bAyLKEaWiGx...
297    ZsMPZ58lz_LDgTxh1WVKs-O6uG-1VzhIyZavJBQIh2ePIK...
298    jL7LxPio5DIKUoEdBYnPT6fAaeeM6hYwQw6tEoqssHoe_I...
299    b9zSLMxUFajuetJBA5TrdTt716nAY3pZ4CPkvU8l6ERisA...
Length: 300, dtype: object

In [ ]:
puuid

0      rk9gcmTmQOHdzPP9ANWHvjB0lmnt0zinvTAVQEgvTNQafk...
1      pFzvA0I0xYkLAJv6hBrwMi-k5GeayGgD7XjPfvNvN75yZv...
2      vnXlUq8lf62uLcBEm4iqKEVt2eFDXa6xbrcgmDaexACM9E...
3      hmiQGBTmq6f0Jm4GNYBGR6IC18xlMSGX5SiJOwdHhbJIqk...
4      2xwCj4UlDQaaqKqk8-Qb1HEyfqW08hJb2ZqfVmMSI_oNKl...
                             ...                        
295    t1_RuefvjeSnVrg0gTmg4-KkSRy8rAeH8CCZVpjQWj2wDp...
296    j4wS7ToUvpSdmkReE1hOcYTKcF4DeaNfF94bAyLKEaWiGx...
297    ZsMPZ58lz_LDgTxh1WVKs-O6uG-1VzhIyZavJBQIh2ePIK...
298    jL7LxPio5DIKUoEdBYnPT6fAaeeM6hYwQw6tEoqssHoe_I...
299    b9zSLMxUFajuetJBA5TrdTt716nAY3pZ4CPkvU8l6ERisA...
Length: 300, dtype: object

In [ ]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

def get_matchid(puuid, key, n, region='asia'):
    matchid = []
    j = 100
    for i in puuid:
        j += 1
        if j&100 ==0:
            sleep(120)
        try:
            request = requests.get(f'http://{region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{i}/ids?count={n}&api_key={key}', headers=request_headers)
            request = json.loads(request.content)
            matchid.extend(request)
        except:
            pass
    
    return list(set(matchid))


ch_matchid = get_matchid(puuid, api_key, 200)
ch_matchid

[]

In [ ]:
ch_matchid

[]

In [ ]:
# 나중에 다시 전부 가져오려면 힘드니 작업 디렉토리에 저장
import pickle
with open('ch_matchid_221123', 'wb') as f:
    pickle.dump(ch_matchid, f)

In [ ]:
# 다시 불러오려면 요렇게
with open('ch_matchid_221123', 'rb') as f:
    list_variable = pickle.load(f)

In [ ]:
game_record = pd.DataFrame()

def match_info(matchid, key, region='asia'):
    l=100
    global game_record
    for i in matchid:
        l+=1
        if l%100 == 0:
            sleep(120)
        try:
            request = requests.get(f'https://{region}.api.riotgames.com/tft/match/v1/matches/{i}?api_key={key}')
            request = json.loads(request.content)
            request = json_normalize(request)
            game_record = pd.concat([game_record,request])
        except:
            pass
    return game_record

In [ ]:
temp_ch = match_info(ch_matchid, api_key)
temp_ch

""


In [ ]:
temp_ch

""


In [ ]:
temp_ch.isnull().sum()

Series([], dtype: float64)

In [ ]:
# 반환받은 에러코드 종류
temp_ch['status.status_code'].unique()

KeyError: ignored

In [ ]:
# 64개 정도의 에러코드는 무시하고, 에러코드가 빈 데이터들만 가져옴
ch_match = temp_ch[temp_ch['status.status_code'].isnull()]

KeyError: ignored

In [ ]:
# 역시 받아오는데 오래 걸리기 때문에, csv형태로 재빨리 저장
temp_ch.to_csv('ch_match_record_221124.csv')

In [ ]:
# 데이터 살펴보기
json_normalize(temp_ch['info.participants'])

KeyError: ignored

In [ ]:
df = json_normalize(json_normalize(temp_ch['info.participants']).iloc[2])
df.columns

In [ ]:
# 'augments'는 증강체, 'traits'는 시너지이며 
# info-tft_game_type이 'pairs(더블업)'일 경우 들어있는 데이터가 조금 다를 수 있다.